<a href="https://colab.research.google.com/github/MWilliamsDP/AlphabetSoupChallenge/blob/main/AlphabetSoup1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [3]:
import io
charity = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))
charity.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
charity.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [8]:
# Determine the number of unique values in each column.
charity.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
typeCounts=charity['APPLICATION_TYPE'].value_counts()
typeCounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=typeCounts[typeCounts < 100].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    charity['APPLICATION_TYPE'] = charity['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
charity['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
classification= charity['CLASSIFICATION'].value_counts()
classification.head(100)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C1283        1
C2500        1
C1248        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [18]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification1=classifcation[classifcation>1]
classification1.head(100)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C4100        6
C1720        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1234        2
C3200        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [30]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification1[classification1 <1000].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    charity['CLASSIFICATION'] = charity['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
charity['CLASSIFICATION'].value_counts()

C1000    17326
Other    16973
Name: CLASSIFICATION, dtype: int64

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`


CatEncoded=pd.get_dummies(charity, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE','ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
CatEncoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [50]:
# Split our preprocessed data into our features and target arrays
y=CatEncoded['IS_SUCCESSFUL'].values
X=CatEncoded.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=48)
X_train

array([[     1,   5000,      0, ...,      0,      1,      0],
       [     1,   5000,      0, ...,      0,      1,      0],
       [     1, 230189,      0, ...,      0,      1,      0],
       ...,
       [     1,   5000,      0, ...,      0,      1,      0],
       [     1,   5000,      0, ...,      0,      1,      0],
       [     1,  17649,      0, ...,      0,      1,      0]])

In [51]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [52]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputFeatures=len(X_train[0])
hiddenLayer1=10
hiddenLayer2=5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hiddenLayer1, input_dim=inputFeatures, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hiddenLayer2, activation ='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation ='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                410       
                                                                 
 dense_7 (Dense)             (None, 5)                 55        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=99)

Epoch 1/99
804/804 [==============================] - 2s 2ms/step - loss: 0.6092 - accuracy: 0.7080
Epoch 2/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5829 - accuracy: 0.7226
Epoch 3/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5763 - accuracy: 0.7248
Epoch 4/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5730 - accuracy: 0.7263
Epoch 5/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5713 - accuracy: 0.7272
Epoch 6/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5699 - accuracy: 0.7263
Epoch 7/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5690 - accuracy: 0.7282
Epoch 8/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5684 - accuracy: 0.7272
Epoch 9/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5680 - accuracy: 0.7281
Epoch 10/99
804/804 [==============================] - 1s 2ms/step - loss: 0.5677 - accuracy: 0.7279

In [55]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5686 - accuracy: 0.7264 - 456ms/epoch - 2ms/step
Loss: 0.5686147212982178, Accuracy: 0.7264139652252197


In [56]:
# Export our model to HDF5 file
nn.save('AlphaetSoupCharity.h5')